# Coordle Backend

In [1]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
from coordle.utils import clean_text
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain
import pymongo


import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/jonavin/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

In [4]:
# Load the last trained model
model = Word2Vec.load(join_path('data', 'cord-19-w2v.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [5]:
import coordle
reload(coordle)
reload(coordle.backend)
reload(coordle.utils)
reload(coordle.preprocessing)
from coordle.backend import (CordDoc, Index, RecursiveDescentParser, 
                             QueryAppenderIndex)


ai_index = QueryAppenderIndex(model.wv.most_similar, n_similars=1)
ai_index.build_from_df(
    df[:16],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 16/16 [00:00<00:00, 367.55it/s]


In [6]:
import coordle
reload(coordle)
from coordle.backend import CordDoc, Index, RecursiveDescentParser, QueryAppenderIndex

docs, scores, errmsgs = ai_index.search('virus NOT white cat')

n = 69
if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')


deque(['virus', 'OR', 'viral', 'NOT', 'white', 'OR', 'cat', 'OR', 'dog'])
1wswi7us  Relationship of SARS-CoV to other pathogenic RNA viruses explored by t  36.8499
qj4dh6rg  Cloaked similarity between HIV-1 and SARS-CoV suggests an anti-SARS st  18.4618
xqhn0vbp  Airborne rhinovirus detection and effect of ultraviolet irradiation on  14.1420
ng4rrdte  Pro/con clinical debate: Steroids are a key component in the treatment  8.9755
5s6acr7m  The Virus That Changed My World                                         8.0834
gi6uaa83  Discovering human history from stomach bacteria                         7.3727
0qaoam29  A double epidemic model for the SARS propagation                        3.9388
1ul8owic  Air pollution and case fatality of SARS in the People's Republic of Ch  1.1574
fy4w7xz8  Association of HLA class I with severe acute respiratory syndrome coro  0.6953
kuybfc1y  Descriptive review of geographic mapping of severe acute respiratory s  0.3231


In [ ]:
ai_index.docmap['grace']

# Mongo

In [ ]:
from importlib import reload
reload(coordle)
reload(coordle.backend)
from coordle import backend

# from coordle.backend import (CordDoc, Index, RecursiveDescentParser, QueryAppenderIndex)

# ram_index = Index()
ram_index = backend.QueryAppenderIndex(model.wv.most_similar, 1)
ram_index.build_from_df(
    df[:128],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

docs, scores, errmsgs = ram_index.search('virus NOT cat white car AND virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}   {score:.4f}')

In [ ]:
import coordle_mongobackend as cm
reload(cm)

# mongoindex = cm.Index('coordle')
mongoindex = cm.AI_Index('coordle-lite', model.wv.most_similar, 1)
mongoindex.drop_old_collections()
mongoindex.build_from_df(df[:128], 'cord_uid', 'title', 'body_text', 
                         use_multiprocessing=True, workers=-1, verbose=True)

In [ ]:
docs, scores, errmsgs = ram_index.search('virus NOT cat white car AND virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}   {score:.4f}')

In [ ]:
%%time
docs, scores, errmsgs = mongoindex.search('virus')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc}   {score:.4f}')

In [ ]:
wordmap = {word:[doc.uid for doc in docs] for word, docs in ai_index.docmap.items() if len(word) < 100}

In [ ]:
client = pymongo.MongoClient()
db_test = client['coordle']
db_test.wordcounts.count_documents({})

In [ ]:
vals, counts = np.unique(['a','a','a','b','b','b','b','c'], return_counts=True)
print(vals)
print(counts)

In [ ]:
thing = pd.value_counts(['a','a','a','b','b','b','b','c'])
print(thing.index.values)
print(thing.values)

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<center><h2> Junkyard </h2></center>

In [ ]:
raise ValueError

Creating sets from dicts is faster than from lists

In [ ]:
from time import time 
from matplotlib import pyplot as plt 
import numpy as np

dict_time = []
list_time = []
ratios = []

xrange = np.arange(1000,1000000,1000)
B = set(list(range(100000)))

for j in xrange:
    a = list(range(j))

    t0=time()
    A = set(a)
    A | B
    t1_list = time()-t0
    list_time.append(t1_list)
    
    a = {i:None for i in range(j)}
    
    t0=time()
#     A = set(a)
    a.keys() | B
    t1_dict = time()-t0
    dict_time.append(t1_dict)

    ratios.append(t1_list/t1_dict)

In [ ]:
plt.plot(xrange[1:], dict_time[1:], label='dict')
plt.plot(xrange[1:], list_time[1:], label='list')
plt.legend()
plt.show()

In [ ]:
plt.plot(xrange[1:], ratios[1:])
plt.show()

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(rdp.parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    